# Evaluate Models for Alternative Decision Thresholds (for AutoMLPipe-BC)
Allows users to specify alternative decision thresholds (rather than the standard .5 probability of case) and re-evaluate algorithm performane metrics 

Designed to operate following application of pipeline phases 1-6. Relies on folder/file hierarchy saved by the pipeline.
 

## Import Packages

In [1]:
import os
import pandas as pd
import pickle
from statistics import mean,stdev
import numpy as np
from scipy import interp,stats
#Evalutation metrics
from sklearn.metrics import accuracy_score
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve, auc, precision_recall_curve
from sklearn import metrics

import warnings
warnings.filterwarnings('ignore')

# Jupyter Notebook Hack: This code ensures that the results of multiple commands within a given cell are all displayed, rather than just the last. 
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

## Set Run Parameters

In [2]:
#experiment_path = "C:/Users/ryanu/Documents/Analysis/AutoMLPipe_Experiments/hcc_demo"
experiment_path = "C:/Users/ryanurb/Documents/Analysis/AutoMLPipe_Experiments/hcc_demo"
targetDataName = 'None' # 'None' if user wants to generate visualizations for all analyzed datasets
algorithms = [] #use empty list if user wishes re-evaluate all modeling algorithms that were run in pipeline.
threshold = 0.3 # Threshold of case probability used to predict case (typically 0.5 by default in modeling)
#available_algorithms = ['Naive Bayes','Logistic Regression','Decision Tree','Random Forest','Gradient Boosting','XGB','LGB','SVM','ANN','K Neighbors','eLCS','XCS','ExSTraCS']

## Automatically detect data folder names

In [3]:
# Get dataset paths for all completed dataset analyses in experiment folder
datasets = os.listdir(experiment_path)
experiment_name = experiment_path.split('/')[-1] #Name of experiment folder
datasets.remove('metadata.csv')
datasets.remove('jobsCompleted')
try:
    datasets.remove('logs')
    datasets.remove('jobs')
except:
    pass
try:
    datasets.remove('DatasetComparisons') #If it has been run previously (overwrite)
except:
    pass
try:
    datasets.remove('KeyFileCopy') #If it has been run previously (overwrite)
except:
    pass
try:
    datasets.remove(experiment_name+'_ML_Pipeline_Report.pdf') #If it has been run previously (overwrite)
except:
    pass
datasets = sorted(datasets) #ensures consistent ordering of datasets
print("Analyzed Datasets: "+str(datasets))

Analyzed Datasets: ['hcc-data_example', 'hcc-data_example_no_covariates']


## Load other necessary parameters

In [4]:
#Load variables specified earlier in the pipeline from metadata file
jupyterRun = 'True'
metadata = pd.read_csv(experiment_path + '/' + 'metadata.csv').values
class_label = metadata[0, 1]
instance_label = metadata[1, 1]
cv_partitions = int(metadata[6,1])
do_NB = metadata[19,1]
do_LR = metadata[20,1]
do_DT = metadata[21,1]
do_RF = metadata[22,1]
do_GB = metadata[23, 1]
do_XGB = metadata[24,1]
do_LGB = metadata[25,1]
do_SVM = metadata[26,1]
do_ANN = metadata[27,1]
do_KN = metadata[28, 1]
do_eLCS = metadata[29,1]
do_XCS = metadata[30,1]
do_ExSTraCS = metadata[31,1]
primary_metric = metadata[32,1]

possible_algos = ['Naive Bayes','Logistic Regression','Decision Tree','Random Forest','Gradient Boosting','XGB','LGB','SVM','ANN','K Neighbors','eLCS','XCS','ExSTraCS']
abbrev = {'Naive Bayes':'NB','Logistic Regression':'LR','Decision Tree':'DT','Random Forest':'RF','Gradient Boosting':'GB','XGB':'XGB','LGB':'LGB','SVM':'SVM','ANN':'ANN','K Neighbors':'KN','eLCS':'eLCS','XCS':'XCS','ExSTraCS':'ExSTraCS'}
colors = {'Naive Bayes':'grey','Logistic Regression':'black','Decision Tree':'yellow','Random Forest':'orange','Gradient Boosting':'bisque','XGB':'purple','LGB':'aqua','SVM':'blue','ANN':'red','eLCS':'firebrick','XCS':'deepskyblue','K Neighbors':'seagreen','ExSTraCS':'lightcoral'}

#Create algorithms list (i.e. modeling algorithms that were run in the pipeline)
if eval(do_NB):
    algorithms.append('Naive Bayes')
if eval(do_LR):
    algorithms.append('Logistic Regression')
if eval(do_DT):
    algorithms.append('Decision Tree')
if eval(do_RF):
    algorithms.append('Random Forest')
if eval(do_GB):
    algorithms.append('Gradient Boosting')
if eval(do_XGB):
    algorithms.append('XGB')
if eval(do_LGB):
    algorithms.append('LGB')
if eval(do_SVM):
    algorithms.append('SVM')
if eval(do_ANN):
    algorithms.append('ANN')
if eval(do_KN):
    algorithms.append('K Neighbors')
if eval(do_eLCS):
    algorithms.append('eLCS')
if eval(do_XCS):
    algorithms.append('XCS')
if eval(do_ExSTraCS):
    algorithms.append('ExSTraCS')
    

## Define necessary methods

In [5]:
def classEval(y_true, y_pred):
    """ Calculates standard classification metrics including:
    True positives, false positives, true negative, false negatives, standard accuracy, balanced accuracy
    recall, precision, f1 score, negative predictive value, likelihood ratio positive, and likelihood ratio negative"""
    #Calculate true positive, true negative, false positive, and false negative.
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    #Calculate Accuracy metrics
    ac = accuracy_score(y_true, y_pred)
    bac = balanced_accuracy_score(y_true, y_pred)
    #Calculate Precision and Recall
    re = recall_score(y_true, y_pred)
    pr = precision_score(y_true, y_pred)
    #Calculate F1 score
    f1 = f1_score(y_true, y_pred)
    # Calculate specificity
    if tn == 0 and fp == 0:
        sp = 0
    else:
        sp = tn / float(tn + fp)
    # Calculate Negative predictive value
    if tn == 0 and fn == 0:
        npv = 0
    else:
        npv = tn/float(tn+fn)
    # Calculate likelihood ratio postive
    if sp == 1:
        lrp = 0
    else:
        lrp = re/float(1-sp)
    # Calculate likeliehood ratio negative
    if sp == 0:
        lrm = 0
    else:
        lrm = (1-re)/float(sp)
    return [bac, ac, f1, re, sp, pr, tp, tn, fp, fn, npv, lrp, lrm]

In [6]:
def primaryStats(algorithms,original_headers,cv_partitions,full_path,data_name,instance_label,class_label,abbrev,colors,plot_ROC,plot_PRC,jupyterRun,name_modifier,legend_inside_plot):
    """ Combine classification metrics and model feature importance scores"""
    result_table = []
    metric_dict = {}
    for algorithm in algorithms: #completed for each individual ML modeling algorithm
        alg_result_table = [] #stores values used in ROC and PRC plots
        # Define evaluation stats variable lists
        s_bac = [] # balanced accuracies
        s_ac = [] # standard accuracies
        s_f1 = [] # F1 scores
        s_re = [] # recall values
        s_sp = [] # specificities
        s_pr = [] # precision values
        s_tp = [] # true positives
        s_tn = [] # true negatives
        s_fp = [] # false positives
        s_fn = [] # false negatives
        s_npv = [] # negative predictive values
        s_lrp = [] # likelihood ratio positive values
        s_lrm = [] # likelihood ratio negative values
        # Define ROC plot variable lists
        tprs = [] # true postitive rates
        aucs = [] #areas under ROC curve
        mean_fpr = np.linspace(0, 1, 100) #used to plot all CVs in single ROC plot
        mean_recall = np.linspace(0, 1, 100) #used to plot all CVs in single PRC plot
        # Define PRC plot variable lists
        precs = [] #precision values for PRC
        praucs = [] #area under PRC curve
        aveprecs = [] #average precisions for PRC
        
        #Gather statistics over all CV partitions
        for cvCount in range(0,cv_partitions):
            #Unpickle saved metrics from previous phase
            result_file = full_path+'/training/'+abbrev[algorithm]+"_CV_"+str(cvCount)+"_metrics"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()
            #Separate pickled results
            metricList = results[0]
            fpr = results[1]
            tpr = results[2]
            roc_auc = results[3]
            prec = results[4]
            recall = results[5]
            prec_rec_auc = results[6]
            ave_prec = results[7]
            #Separate metrics from metricList
            s_bac.append(metricList[0])
            s_ac.append(metricList[1])
            s_f1.append(metricList[2])
            s_re.append(metricList[3])
            s_sp.append(metricList[4])
            s_pr.append(metricList[5])
            s_tp.append(metricList[6])
            s_tn.append(metricList[7])
            s_fp.append(metricList[8])
            s_fn.append(metricList[9])
            s_npv.append(metricList[10])
            s_lrp.append(metricList[11])
            s_lrm.append(metricList[12])
            #update list that stores values used in ROC and PRC plots
            alg_result_table.append([fpr, tpr, roc_auc, recall, prec, prec_rec_auc, ave_prec])
            # Update ROC plot variable lists needed to plot all CVs in one ROC plot
            tprs.append(interp(mean_fpr, fpr, tpr))
            tprs[-1][0] = 0.0
            aucs.append(roc_auc)
            # Update PRC plot variable lists needed to plot all CVs in one PRC plot
            precs.append(interp(mean_recall, recall, prec))
            praucs.append(prec_rec_auc)
            aveprecs.append(ave_prec)
            
        if jupyterRun:
            print(algorithm)
        #Define values for the mean ROC line (mean of individual CVs)
        mean_tpr = np.mean(tprs, axis=0)
        mean_tpr[-1] = 1.0
        mean_auc = np.mean(aucs)
        #Generate ROC Plot (including individual CV's lines, average line, and no skill line) - based on https://scikit-learn.org/stable/auto_examples/model_selection/plot_roc_crossval.html-----------------------
        if eval(plot_ROC):
            # Set figure dimensions
            plt.rcParams["figure.figsize"] = (6,6)
            # Plot individual CV ROC lines
            for i in range(cv_partitions):
                plt.plot(alg_result_table[i][0], alg_result_table[i][1], lw=1, alpha=0.3,label='ROC fold %d (AUC = %0.3f)' % (i, alg_result_table[i][2]))
            # Plot no-skill line
            plt.plot([0, 1], [0, 1], linestyle='--', lw=2, color='r',label='No-Skill', alpha=.8)
            # Plot average line for all CVs
            std_auc = np.std(aucs) # AUC standard deviations across CVs
            plt.plot(mean_fpr, mean_tpr, color=colors[algorithm],label=r'Mean ROC (AUC = %0.3f $\pm$ %0.3f)' % (mean_auc, std_auc),lw=2, alpha=.8)
            # Plot standard deviation grey zone of curves
            std_tpr = np.std(tprs, axis=0)
            tprs_upper = np.minimum(mean_tpr + std_tpr, 1)
            tprs_lower = np.maximum(mean_tpr - std_tpr, 0)
            plt.fill_between(mean_fpr, tprs_lower, tprs_upper, color='grey', alpha=.2,label=r'$\pm$ 1 std. dev.')
            #Specify plot axes,labels, and legend
            plt.xlim([-0.05, 1.05])
            plt.ylim([-0.05, 1.05])
            plt.xlabel('False Positive Rate')
            plt.ylabel('True Positive Rate')
            if legend_inside_plot:
                plt.legend(loc="lower right")
            else:
                plt.legend(loc="upper left", bbox_to_anchor=(1.01,1))
            #Export and/or show plot
            plt.savefig(full_path+'/training/results/'+abbrev[algorithm]+"_ROC"+name_modifier+".png", bbox_inches="tight")
            if eval(jupyterRun):
                plt.show()
            else:
                plt.close('all')

        #Define values for the mean PRC line (mean of individual CVs)
        mean_prec = np.mean(precs, axis=0)
        mean_pr_auc = np.mean(praucs)
        #Generate PRC Plot (including individual CV's lines, average line, and no skill line)------------------------------------------------------------------------------------------------------------------
        if eval(plot_PRC):
            # Set figure dimensions
            plt.rcParams["figure.figsize"] = (6,6)
            # Plot individual CV PRC lines
            for i in range(cv_partitions):
                plt.plot(alg_result_table[i][3], alg_result_table[i][4], lw=1, alpha=0.3, label='PRC fold %d (AUC = %0.3f)' % (i, alg_result_table[i][5]))
            #Estimate no skill line based on the fraction of cases found in the first test dataset
            test = pd.read_csv(full_path + '/CVDatasets/' + data_name + '_CV_0_Test.csv') #Technically there could be a unique no-skill line for each CV dataset based on final class balance (however only one is needed, and stratified CV attempts to keep partitions with similar/same class balance)
            testY = test[class_label].values
            noskill = len(testY[testY == 1]) / len(testY)  # Fraction of cases
            # Plot no-skill line
            plt.plot([0, 1], [noskill, noskill], color='orange', linestyle='--', label='No-Skill', alpha=.8)
            # Plot average line for all CVs
            std_pr_auc = np.std(praucs)
            # Plot standard deviation grey zone of curves
            plt.plot(mean_recall, mean_prec, color=colors[algorithm],label=r'Mean PRC (AUC = %0.3f $\pm$ %0.3f)' % (mean_pr_auc, std_pr_auc),lw=2, alpha=.8)
            std_prec = np.std(precs, axis=0)
            precs_upper = np.minimum(mean_prec + std_prec, 1)
            precs_lower = np.maximum(mean_prec - std_prec, 0)
            plt.fill_between(mean_fpr, precs_lower, precs_upper, color='grey', alpha=.2,label=r'$\pm$ 1 std. dev.')
            #Specify plot axes,labels, and legend
            plt.xlim([-0.05, 1.05])
            plt.ylim([-0.05, 1.05])
            plt.xlabel('Recall (Sensitivity)')
            plt.ylabel('Precision (PPV)')
            if legend_inside_plot:
                plt.legend(loc="upper right")
            else:
                plt.legend(loc="upper left", bbox_to_anchor=(1.01,1))
            #Export and/or show plot
            plt.savefig(full_path+'/training/results/'+abbrev[algorithm]+"_PRC"+name_modifier+".png", bbox_inches="tight")
            if eval(jupyterRun):
                plt.show()
            else:
                plt.close('all')
                
        #Export and save all CV metric stats for each individual algorithm  -----------------------------------------------------------------------------
        results = {'Balanced Accuracy': s_bac, 'Accuracy': s_ac, 'F1_Score': s_f1, 'Sensitivity (Recall)': s_re, 'Specificity': s_sp,'Precision (PPV)': s_pr, 'TP': s_tp, 'TN': s_tn, 'FP': s_fp, 'FN': s_fn, 'NPV': s_npv, 'LR+': s_lrp, 'LR-': s_lrm, 'ROC_AUC': aucs,'PRC_AUC': praucs, 'PRC_APS': aveprecs}
        metric_dict[algorithm] = results

        #Store ave metrics for creating global ROC and PRC plots later
        mean_ave_prec = np.mean(aveprecs)
        result_dict = {'algorithm':algorithm,'fpr':mean_fpr, 'tpr':mean_tpr, 'auc':mean_auc, 'prec':mean_prec, 'pr_auc':mean_pr_auc, 'ave_prec':mean_ave_prec}
        result_table.append(result_dict)
    #Result table later used to create global ROC an PRC plots comparing average ML algorithm performance.
    result_table = pd.DataFrame.from_dict(result_table)
    result_table.set_index('algorithm',inplace=True)

    return result_table,metric_dict


## Generate ROC and PRC plots

In [14]:
if not targetDataName == 'None': # User specified one analyzed dataset above (if more than one were analyzed)
    for each in datasets:
        if not each == targetDataName:
            datasets.remove(each)
    print("Vizualized Datasets: "+str(datasets))

for each in datasets: #each analyzed dataset to make plots for
    print("---------------------------------------")
    print(each)
    print("---------------------------------------")
    full_path = experiment_path+'/'+each
    #Create folder for tree vizualization files
    original_headers = pd.read_csv(full_path+"/exploratory/OriginalHeaders.csv",sep=',').columns.values.tolist() #Get Original Headers

    for algorithm in algorithms: #loop through algorithms
        print(algorithm)
        for cvCount in range(0,cv_partitions): #loop through cv's
            print(str(cvCount))
            #load testing data
            test_file_path = full_path + '/CVDatasets/' + each + "_CV_" + str(cvCount) + "_Test.csv"
            test = pd.read_csv(test_file_path)
            testY = test[class_label].values
            del test #memory cleanup
            
            #Load pickled metric file for given algorithm and cv
            result_file = full_path+'/training/'+abbrev[algorithm]+"_CV_"+str(cvCount)+"_metrics"
            file = open(result_file, 'rb')
            results = pickle.load(file)
            file.close()

            #load probas_ for model file
            probas_ = results[9]

            
            #Get new class predictions given specified decision threshold
            y_pred = probas_[:,1] > 0.000001 #threshold

            integer_map = map(int, y_pred) 
            integer_list = list(integer_map)


            #Calculate standard classificaction metrics
            metricList = classEval(testY, y_pred)
 
            

            #add to metric list

            #create new cv metric file for each algorithm
    
    #create new average metric file 
    
    
    #result_table,metric_dict = primaryStats(algorithms,original_headers,cv_partitions,full_path,each,instance_label,class_label,abbrev,colors,plot_ROC,plot_PRC,jupyterRun,name_modifier,legend_inside_plot)

 

---------------------------------------
hcc-data_example
---------------------------------------
Naive Bayes
0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1.]
[[9.97406489e-01 2.59351142e-03]
 [9.99102336e-01 8.97663652e-04]
 [9.99999191e-01 8.08525595e-07]
 [1.00000000e+00 2.03323992e-15]
 [9.99999998e-01 1.77195505e-09]
 [9.99999973e-01 2.66918833e-08]
 [9.99225002e-01 7.74997624e-04]
 [9.99180203e-01 8.19797076e-04]
 [9.99999982e-01 1.76963061e-08]
 [9.99998485e-01 1.51478647e-06]
 [9.99631896e-01 3.68103723e-04]
 [9.99999998e-01 1.55873726e-09]
 [9.99999061e-01 9.39434296e-07]
 [9.99999989e-01 1.14043636e-08]
 [9.99999997e-01 3.44649330e-09]
 [9.99999999e-01 1.27951549e-09]
 [9.99999991e-01 8.67450989e-09]
 [9.99999999e-01 5.01298309e-10]
 [9.99999740e-01 2.59643505e-07]
 [9.99999889e-01 1.11343288e-07]
 [9.99999995e-01 4.91032464e-09]
 [9.99711362e-01 2.88637603